In [ ]:
full_tables = [
    'dimension_city',
    'dimension_customer',
    'dimension_date',
    'dimension_employee',
    'dimension_stock_item',
    'fact_sale_1y_full'
    ]

In [ ]:
from pyspark.sql.functions import col, year, month, quarter

path = "wasbs://sampledata@azuresynapsestorage.blob.core.windows.net/WideWorldImportersDW/parquet/full/"

def load_fulldatafromsource(table_name):
  file_location = path + table_name
  df = spark.read.format("parquet").load(file_location)  
  if "fact_sale" in table_name: 
      table_name = "fact_sale"
      
      print("loading full data for table " + table_name + "...")
      df = df.withColumn('Year', year(col("InvoiceDateKey")))
      df = df.withColumn('Quarter', quarter(col("InvoiceDateKey")))
      df = df.withColumn('Month', month(col("InvoiceDateKey")))
      df.write.mode("overwrite").format("parquet").partitionBy("Year","Quarter").save("Files/bronze/" + table_name)

      print("loaded full data for table " + table_name + "!")
  else:
      print("loading full data for table " + table_name + "...")
      df.write.mode("overwrite").format("parquet").save("Files/bronze/" + table_name)
      print("loaded full data for table " + table_name + "!")   

In [ ]:
for table in full_tables:
    load_fulldatafromsource(table)

In [ ]:
def create_tables(table_name):
    if "fact_sale" in table_name: 
      table_name = "fact_sale"

    df = spark.read.format("parquet").load('Files/bronze/' + table_name)
    df.write.mode("overwrite").format("delta").save("Tables/wwi_" + table_name)

In [ ]:
for table in full_tables:
    create_tables(table)